#Using Word2Vec, FastText embedding and nearest neighbors with Annoy

In [11]:
!pip install cleanco transliterate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.9 MB/s 


In [34]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import os
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import random
import time
import re
import unicodedata
from cleanco import basename
from transliterate.decorators import transliterate_function

In [3]:
#загрузка данных с yandex disk
!wget -O train.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/03C9AjRJqukWcg

--2022-10-26 18:39:20--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/03C9AjRJqukWcg
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/87460a5b970639ba9e84d4f25b3d4754cb84a0860cb5a717bef0f42958ad73f9/6359b451/msMO0KQIAH3VfHbAiTRYc7IU3Nhh3xJ0wfmoVk6qNzxYgqaBMRY0efTvRnMIfEhMPEeBAfXl4uChRAP2dvlWOg%3D%3D?uid=0&filename=train_brands.csv&disposition=attachment&hash=Q%2BL0DOkc2P3%2BfmzFCyxRcXa2Pi%2B4abC8b81eIqanwFGKgEtQzyjIoJ0TbqC7RIGwq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fplain&owner_uid=6709741&fsize=29759004&hid=d3075775f2ccb8067d5c2aa80ed316de&media_type=spreadsheet&tknv=v2 [following]
--2022-10-26 18:39:21--  https://downloader.disk.yandex.ru/disk/87460a5b970639ba9e84d4f25b3d4754cb84a0860cb5a717bef0f42958ad73f9/6359b451/msMO0KQIAH3VfHbAiTRYc7IU3N

In [4]:
train_path = 'train.csv'

In [5]:
train_df = pd.read_csv(train_path)
train_df

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...,...
497814,497815,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497815,497816,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497816,497817,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0
497817,497818,"Shanghai Kechuan Trading Co., Ltd.",Shanghai M&G Stationery Inc.,0


In [6]:
train_df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [7]:
train_df.is_duplicate.value_counts(normalize=True)*100

0    99.265195
1     0.734805
Name: is_duplicate, dtype: float64

In [ ]:
train_df[train_df.is_duplicate==1][:20]

In [ ]:
train_df[train_df.is_duplicate==0].tail(20)

In [17]:
a = ord('а') # cyrillic "А"
rus_alphabet = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])

In [21]:
def preprocessing(x: str) -> str:
    
    def rus_preprocess(x: str) -> str:
        forms_of_ownership = [
            'ооо',
            'оао',
            'общество с ограниченной ответственностью',
            'открытое акционерное общество',
            'филиал компании'
        ]
        
        for form in forms_of_ownership:
            pattern = re.compile(form)
            x = pattern.sub('', x)
            
        return x
    
    @transliterate_function(language_code='ru', reversed=True)
    def translit(x: str) -> str:
        return x
    
    
    x = x.strip().lower()
    x = basename(x) if not bool(set(rus_alphabet).intersection(set(x.lower()))) else rus_preprocess(x)
    x = unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode() if not bool(set(rus_alphabet).intersection(set(x.lower()))) else translit(x)
    x = basename(x)
    x = re.sub(r'[^\w\s]',' ', x)
    
    return [s for s in x.split()] if len(x.split()) != 0 else '-' * 5

In [13]:
name_1_corpus = train_df.name_1
name_2_corpus = train_df.name_2

In [35]:
print(len(name_1_corpus))
index = random.randint(0, train_df.shape[0])
name_1_corpus[index]

497819


'Darshan Kripa General Trading Llc'

In [22]:
n_1_all=[]
for line in tqdm_notebook(name_1_corpus):
        spls = preprocessing(line)
        n_1_all.append(spls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/497819 [00:00<?, ?it/s]

In [36]:
print(len(n_1_all))
n_1_all[index]

497819


['darshan', 'kripa', 'general', 'trading']

In [24]:
n_2_all=[]
for line in tqdm_notebook(name_2_corpus):
        spls = preprocessing(line)
        n_2_all.append(spls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/497819 [00:00<?, ?it/s]

#Word2Vec, Fasttext, Annoy

In [25]:
#create corpus
all_names = n_1_all+n_2_all

In [26]:
# training models (takes some time)

modelW2V = Word2Vec(sentences=all_names, size=100, window=5, min_count=1)

modelFT = FastText(sentences=all_names, size=100, min_count=1, window=5, workers=8)

In [27]:
# craete Annoy indexes
index_w2v = annoy.AnnoyIndex(100 ,'angular') #using angular metric
index_ft = annoy.AnnoyIndex(100 ,'angular')


for i, sent in enumerate(all_names):

    res1 = [modelW2V.wv[w] for w in sent if w in modelW2V.wv]
    v_w2v = sum(res1) / len(res1) if res1 else np.zeros(100)

    res2 = [modelFT.wv[w] for w in sent if w in modelFT.wv]
    v_ft = sum(res2) / len(res2) if res2 else np.zeros(100)

    index_w2v.add_item(i, v_w2v)
    index_ft.add_item(i, v_ft)



index_w2v.build(10) #10 Trees
index_ft.build(10)

True

In [28]:
names_raw = train_df.name_1 + train_df.name_2

In [29]:
def get_response(text, index, model):
    text = preprocessing(text)
    w_v  = [model.wv[w] for w in text if w in model.wv ]
    tweet_vector = sum(w_v) / len(w_v) if w_v else np.zeros(100)
    result = index.get_nns_by_vector(tweet_vector, 10) #10 nearest with distance

    return [names_raw[i] for i in result] 

In [37]:
# choose random brand
df_dub = train_df[train_df.is_duplicate==1]
rand_i = random.randint(0, df_dub.shape[0])
test_brand = df_dub.name_1.iloc[rand_i]
print(f'исходная компания {test_brand}')
print('----------------')
print(f'соответствие из исходной таблицы {df_dub.name_2.iloc[rand_i]}')

исходная компания Pt Bridgestone Tire Indonesia
----------------
соответствие из исходной таблицы Bridgestone India


In [39]:
# word2vec vectors
%time
response = get_response(test_brand, index_w2v, modelW2V)
print(test_brand)
print('*'*50)
for line in response:
  
  print(line)
  print('-'*50)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
Pt Bridgestone Tire Indonesia
**************************************************
Pt Bridgestone Tire IndonesiaBridgestone Do Brasil Industria E Comenrcio Ltda
--------------------------------------------------
Pt Bridgestone Tire IndonesiaThai Bridgestone Co., Ltd.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaUsa.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Firestone Venezolana C
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Do Brasil Industria E Comercio Ltda
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Canada Inc.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone India Pvt., Ltd.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Firestone Do Brasil
-----------------

In [38]:
#FastText vectors
%time
response = get_response(test_brand, index_ft, modelFT)
print(test_brand)
print('*'*50)
for line in response:
  
  print(line)
  print('-'*50)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 26 µs
Pt Bridgestone Tire Indonesia
**************************************************
Pt Bridgestone Tire IndonesiaBridgestone Do Brasil Industria E Comenrcio Ltda
--------------------------------------------------
Pt Bridgestone Tire IndonesiaThai Bridgestone Co., Ltd.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaUsa.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Firestone Venezolana C
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Do Brasil Industria E Comercio Ltda
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Canada Inc.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone India Pvt., Ltd.
--------------------------------------------------
Pt Bridgestone Tire IndonesiaBridgestone Firestone Do Brasil
-----------------

#both models showed good results, the processing speed of each name is equal, which allows you to scale the project